In [21]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import time

from Qulacs_QAOA import Qulacs_QAOA
from src.Chain import Chain
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       qubo_limits, 
                       check_qubo)

In [22]:

N = 15
seed = 100
 # Choosing optimizer for scipy
optimizer_method = 'COBYLA'
k = N // 2
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(N)])
my_indices = [(i, i+1) for i in range(N-1)]
n_layers = 4
alpha = 0.5
max_iter = 10
 # Defining topology
my_chain = Chain(N_qubits=N)
my_chain.set_initialization_strategy(strategy=init_strat)
# Deciding between grid and 1d chain topology
my_topology = my_chain
# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=N, seed=seed)
# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)
portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])
# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])

if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')
         

In [23]:
qulacs_ansatz = Qulacs_QAOA(N_qubits=N,
                            layers=n_layers,
                            QUBO_matrix=Q,
                            QUBO_offset=offset)

In [24]:
theta_i = np.random.uniform(0,1,2*n_layers)

In [36]:
qulacs_ansatz.get_cost(angles=theta_i)


-0.09877608646233413